# 1절. SQLite데이터 베이스 연결


## 1.1 SQLite와 파이썬
- Lite하기 때문에 test용으로 테이터 몇 백개 넣을때 활용. 프로토타입(시제품 나오기 전까지 제품)을 만들 때까지 사용. 정식제품은  ORACLE이나 다른 databse로 이식 
- SQLite 데이터 베이스는 별도의 subprocess 없이 SQL을 사용하여 DB액세스할 수 있도록 만든 / 간단한 disk기반 데이터베이스를 제공하는 / 'C라이브러리'
- DB browser for SQLite (https://sqlitebrowser.org/dl/)


In [7]:
import sqlite3
sqlite3.sqlite_version

'3.33.0'

## 1.2 데이터 베이스 연결

In [8]:
# 파일안에 table를 만들고, table를 액세스

#DB 연결 객체 생성
conn = sqlite3.connect('data/ch10_example.db')  #빈 db file이 하나 만들어짐 
conn

In [3]:
#conn.close()

In [9]:
conn

In [10]:
#커서 객체 반환. 커서는 SQL문을 실행시키고 + 결과 데이터를 조회함
cursor = conn.cursor()

In [16]:
cursor.execute("""
        CREATE TABLE MEMBER(
            NAME TEXT,
            AGE INT,
            EMAIL TEXT
        )
""")

In [ ]:
# DB Broswer for SQLite --> 파일  --> 읽기전용으로 데이터 베이스 열기 *************************************
# 데이블 생성하기 

In [14]:
#cursor.execute("DROP TABLE MEMBER")    

In [17]:
cursor.execute("INSERT INTO MEMBER VALUES ('홍길동',20,'h@hong.com')")
cursor.execute("INSERT INTO MEMBER VALUES ('유길동',23,'y@hong.com')")
cursor.execute("INSERT INTO MEMBER VALUES ('마길동',25,'m@hong.com')")

In [18]:
conn.commit() #(반) conn.rollback()

In [25]:
cursor.execute('SELECT * FROM MEMBER') #실행 결과는 cursor가 가르킴 

In [26]:
# select문의 결과를 받는 함수: 
## fetchall() - 결과를 모두 받을 때
## fetchone() -  결과를 한 행씩 받을 때 
## fetchmany(n) - 결과를 n행 받을 때           fetchmany(3)  3줄
print(cursor.fetchall())                      #결과 tuple로 나옴    한행한행 반드시 tuple로 나옴. fetall로해서 list로 나옴 

[('홍길동', 20, 'h@hong.com'), ('유길동', 23, 'y@hong.com'), ('마길동', 25, 'm@hong.com')]


In [27]:
print(cursor.fetchall())  #한번 fetchall를 했기때문에, 변수에 담지 않았으면 아무것도 출력이 안됨 

[]


In [28]:
cursor.execute("SELECT * FROM MEMBER")
members = cursor.fetchall()          #계속 쓰려면 '변수'에 담아야
for member in members:
    print(member)  

('홍길동', 20, 'h@hong.com')
('유길동', 23, 'y@hong.com')
('마길동', 25, 'm@hong.com')


In [29]:
members[0]

('홍길동', 20, 'h@hong.com')

In [30]:
cursor.execute("SELECT * FROM MEMBER")
print(cursor.fetchone())
print(cursor.fetchone())
print(cursor.fetchone())
print(cursor.fetchone())

('홍길동', 20, 'h@hong.com')
('유길동', 23, 'y@hong.com')
('마길동', 25, 'm@hong.com')
None


In [32]:
cursor.execute("SELECT * FROM MEMBER")
while True:
    member = cursor.fetchone()
    if member is None:
        break;
    print(member)


('홍길동', 20, 'h@hong.com')
('유길동', 23, 'y@hong.com')
('마길동', 25, 'm@hong.com')


In [33]:
cursor.execute("SELECT * FROM MEMBER")
for member in cursor.fetchmany(2):
    print(member)

('홍길동', 20, 'h@hong.com')
('유길동', 23, 'y@hong.com')


In [35]:
cursor.execute("SELECT * FROM MEMBER")
for member in cursor.fetchmany(2):
    print(member[0],member[1],member[2])

홍길동 20 h@hong.com
유길동 23 y@hong.com


In [34]:
cursor.fetchmany(2)   #2개 가지고 오라고 했지만, 1개 밖에 안남아서 하나만 가져옴 

[('마길동', 25, 'm@hong.com')]

In [37]:
cursor.close() #생략 가능
conn.close()

## 1.3 SQL구문에 파라미터 사용하기
- qmark, named(추천)방법

In [41]:
"""
conn = sqlite3.connect('data/ch10_example.db')
cursor = conn.cursor()
cursor.execute("SELECT * FROM MEMBER WHERE EMAIL IN ('k@kim.com','h@hong.com')")
print(cursor.fetchall())

"""


[('홍길동', 20, 'h@hong.com')]


- **파라미터 사용하기: qmark 방법 이용**

In [44]:
# 파라미터 사용하기: qmark 방법 이용
conn = sqlite3.connect('data/ch10_example.db')
cursor = conn.cursor()
cursor.execute("SELECT * FROM MEMBER WHERE EMAIL IN (?,?)",('홍길동','김철수'))  # 여기서 '홍길동','김철수'가 PARAMETER
print(cursor.fetchall())

[]


In [45]:
name1= input('검색할 이름1:')
name2= input('검색할 이름2:')
names=(name1,name2)
cursor.execute("SELECT * FROM MEMBER WHERE NAME IN (?,?)",names)
print(cursor.fetchall())

검색할 이름1:홍길동
검색할 이름2:마길동
[('홍길동', 20, 'h@hong.com'), ('마길동', 25, 'm@hong.com')]


- **파라미터 사용하기: named 방법** 

In [51]:
## 파라미터 사용하기: named 방법 
name = input('검색하고자 하는 이름: ')
cursor.execute("SELECT * FROM MEMBER WHERE NAME=:name", {'name':name})
members = cursor.fetchall()
if len(members) ==0:
    print('해당 이름의 데이터가 없습니다')
else:
    print(cursor.fetchall())
print(cursor.fetchall())


검색하고자 하는 이름: f
해당 이름의 데이터가 없습니다
[]


In [ ]:
try:
    name = input('입력할 이름?')
    age = int(input('입력할 나이?'))
    email=input("입력할 메일?")
except:
    print('나이를 잘못 입력하셨어요. 초기값으로 20살')
    age = 20
    email = input('입력할 메일?')

cursor.execute("INSERT INTO MEMBER VALUES(:name,:age,:email)",   ### : 왜 필요하는지???????????????????????????????????????
              {'name':name,'age':age,'email':email}
              )

In [ ]:
cursorexecute('select * from member')
print(cursor.fetchall())

In [ ]:
cursor.close()
conn.close()

# 2절. 오라클 데이터 베이스 연결

- anaconda prompt 실행 --> 입력 내용:pip install cx_Oracle 

In [ ]:
# anaconda prompt 실행 --> 입력 내용:pip install cx_Oracle 

    #제대로 install되었는지 확인법
    #pip show cx_Oracle 

In [53]:
#데이터 베이스 설정(DBMS가 있는 서버주소, 포트번호, sid등을 설정)


'(DESCRIPTION=(ADDRESS=(PROTOCOL=TCP)(HOST=localhost)(PORT=1521))(CONNECT_DATA=(SID=xe)))'

In [54]:
conn = cx_Oracle.connect("scott","tiger",dsn=oracle_dsn)
conn
# 여기서 에러날 시 VC_redist.x64.exe 인스톨 

<cx_Oracle.Connection to scott@(DESCRIPTION=(ADDRESS=(PROTOCOL=TCP)(HOST=localhost)(PORT=1521))(CONNECT_DATA=(SID=xe)))>

In [55]:
#conn얻어오는 방법(1)
oracle_dsn = cx_Oracle.makedsn(host="localhost",port=1521, sid="xe")
conn = cx_Oracle.connect("scott","tiger",dsn=oracle_dsn)

In [56]:
conn.close()

In [57]:
#conn얻어오는 방법(2)
conn = cx_Oracle.connect("scott","tiger","localhost:1521/xe")
conn



<cx_Oracle.Connection to scott@localhost:1521/xe>

In [58]:
# cursor 얻어오기 
cursor = conn.cursor()

In [59]:
sql ="SELECT * FROM EMP"
cursor.execute(sql)
emp = cursor.fetchall()

In [60]:
for e in emp:
    print(e)

(7369, 'SMITH', 'CLERK', 7902, datetime.datetime(1980, 12, 17, 0, 0), 800.0, None, 20)
(7499, 'ALLEN', 'SALESMAN', 7698, datetime.datetime(1981, 2, 20, 0, 0), 1600.0, 300.0, 30)
(7521, 'WARD', 'SALESMAN', 7698, datetime.datetime(1981, 2, 22, 0, 0), 1250.0, 500.0, 30)
(7566, 'JONES', 'MANAGER', 7839, datetime.datetime(1981, 4, 2, 0, 0), 2975.0, None, 20)
(7654, 'MARTIN', 'SALESMAN', 7698, datetime.datetime(1981, 9, 28, 0, 0), 1250.0, 1400.0, 30)
(7698, 'BLAKE', 'MANAGER', 7839, datetime.datetime(1981, 5, 1, 0, 0), 2850.0, None, 30)
(7782, 'CLARK', 'MANAGER', 7839, datetime.datetime(1981, 6, 9, 0, 0), 2450.0, None, 10)
(7788, 'SCOTT', 'ANALYST', 7566, datetime.datetime(1982, 12, 9, 0, 0), 3000.0, None, 20)
(7839, 'KING', 'PRESIDENT', None, datetime.datetime(1981, 11, 17, 0, 0), 5000.0, None, 10)
(7844, 'TURNER', 'SALESMAN', 7698, datetime.datetime(1981, 9, 8, 0, 0), 1500.0, 0.0, 30)
(7876, 'ADAMS', 'CLERK', 7788, datetime.datetime(1983, 1, 12, 0, 0), 1100.0, None, 20)
(7900, 'JAMES', 'CL

In [61]:
#한번에 하는 방법. 위와 같음 
sql ="SELECT * FROM EMP"
cursor.execute(sql)
for e in cursor:  #cursor가 하나하나 증가하면서
    print(e)

(7369, 'SMITH', 'CLERK', 7902, datetime.datetime(1980, 12, 17, 0, 0), 800.0, None, 20)
(7499, 'ALLEN', 'SALESMAN', 7698, datetime.datetime(1981, 2, 20, 0, 0), 1600.0, 300.0, 30)
(7521, 'WARD', 'SALESMAN', 7698, datetime.datetime(1981, 2, 22, 0, 0), 1250.0, 500.0, 30)
(7566, 'JONES', 'MANAGER', 7839, datetime.datetime(1981, 4, 2, 0, 0), 2975.0, None, 20)
(7654, 'MARTIN', 'SALESMAN', 7698, datetime.datetime(1981, 9, 28, 0, 0), 1250.0, 1400.0, 30)
(7698, 'BLAKE', 'MANAGER', 7839, datetime.datetime(1981, 5, 1, 0, 0), 2850.0, None, 30)
(7782, 'CLARK', 'MANAGER', 7839, datetime.datetime(1981, 6, 9, 0, 0), 2450.0, None, 10)
(7788, 'SCOTT', 'ANALYST', 7566, datetime.datetime(1982, 12, 9, 0, 0), 3000.0, None, 20)
(7839, 'KING', 'PRESIDENT', None, datetime.datetime(1981, 11, 17, 0, 0), 5000.0, None, 10)
(7844, 'TURNER', 'SALESMAN', 7698, datetime.datetime(1981, 9, 8, 0, 0), 1500.0, 0.0, 30)
(7876, 'ADAMS', 'CLERK', 7788, datetime.datetime(1983, 1, 12, 0, 0), 1100.0, None, 20)
(7900, 'JAMES', 'CL

In [63]:
sql = "SELECT * FROM EMP WHERE DEPTNO=:deptno"  #named방법
#sql = "SELECT * FROM EMP WHERE DEPTNO=?"     qmark방법 
deptno = input("검색하고자 하는 부서 번호는?")
cursor.execute(sql,{'deptno':deptno})
for emp in cursor:
    print(emp)

검색하고자 하는 부서 번호는?20
(7369, 'SMITH', 'CLERK', 7902, datetime.datetime(1980, 12, 17, 0, 0), 800.0, None, 20)
(7566, 'JONES', 'MANAGER', 7839, datetime.datetime(1981, 4, 2, 0, 0), 2975.0, None, 20)
(7788, 'SCOTT', 'ANALYST', 7566, datetime.datetime(1982, 12, 9, 0, 0), 3000.0, None, 20)
(7876, 'ADAMS', 'CLERK', 7788, datetime.datetime(1983, 1, 12, 0, 0), 1100.0, None, 20)
(7902, 'FORD', 'ANALYST', 7566, datetime.datetime(1981, 12, 3, 0, 0), 3000.0, None, 20)


In [64]:
sql = "SELECT * FROM EMP WHERE DEPTNO=:deptno" 
deptno = input("검색하고자 하는 부서 번호는?")
cursor.execute(sql,{'deptno':deptno})

emp=cursor.fetchall()
if len(emp)==0:
    print("입력하신 부서번호의 데이터가 없습니다.")
else:
    for e in emp:
        print(e)

검색하고자 하는 부서 번호는?30
(7499, 'ALLEN', 'SALESMAN', 7698, datetime.datetime(1981, 2, 20, 0, 0), 1600.0, 300.0, 30)
(7521, 'WARD', 'SALESMAN', 7698, datetime.datetime(1981, 2, 22, 0, 0), 1250.0, 500.0, 30)
(7654, 'MARTIN', 'SALESMAN', 7698, datetime.datetime(1981, 9, 28, 0, 0), 1250.0, 1400.0, 30)
(7698, 'BLAKE', 'MANAGER', 7839, datetime.datetime(1981, 5, 1, 0, 0), 2850.0, None, 30)
(7844, 'TURNER', 'SALESMAN', 7698, datetime.datetime(1981, 9, 8, 0, 0), 1500.0, 0.0, 30)
(7900, 'JAMES', 'CLERK', 7698, datetime.datetime(1981, 12, 3, 0, 0), 950.0, None, 30)


- **데이터 분석시 list,dictionary -->dataframe으로 변환**

In [65]:
print(emp)

[(7499, 'ALLEN', 'SALESMAN', 7698, datetime.datetime(1981, 2, 20, 0, 0), 1600.0, 300.0, 30), (7521, 'WARD', 'SALESMAN', 7698, datetime.datetime(1981, 2, 22, 0, 0), 1250.0, 500.0, 30), (7654, 'MARTIN', 'SALESMAN', 7698, datetime.datetime(1981, 9, 28, 0, 0), 1250.0, 1400.0, 30), (7698, 'BLAKE', 'MANAGER', 7839, datetime.datetime(1981, 5, 1, 0, 0), 2850.0, None, 30), (7844, 'TURNER', 'SALESMAN', 7698, datetime.datetime(1981, 9, 8, 0, 0), 1500.0, 0.0, 30), (7900, 'JAMES', 'CLERK', 7698, datetime.datetime(1981, 12, 3, 0, 0), 950.0, None, 30)]


In [67]:
#dataframe은 pandas에 있다. 따라서 import pandas
import pandas as pd
cursor.execute("SELECT * FROM EMP")
data = cursor.fetchall() #튜플 리스트 
data_df = pd.DataFrame(data) #데이터 프레임 ******************************
data_df



,0,1,2,3,4,5,6,7
0,7369,SMITH,CLERK,7902.0,1980-12-17,800.0,NaN,20
1,7499,ALLEN,SALESMAN,7698.0,1981-02-20,1600.0,300.0,30
2,7521,WARD,SALESMAN,7698.0,1981-02-22,1250.0,500.0,30
3,7566,JONES,MANAGER,7839.0,1981-04-02,2975.0,NaN,20
4,7654,MARTIN,SALESMAN,7698.0,1981-09-28,1250.0,1400.0,30
5,7698,BLAKE,MANAGER,7839.0,1981-05-01,2850.0,NaN,30
6,7782,CLARK,MANAGER,7839.0,1981-06-09,2450.0,NaN,10
7,7788,SCOTT,ANALYST,7566.0,1982-12-09,3000.0,NaN,20
8,7839,KING,PRESIDENT,NaN,1981-11-17,5000.0,NaN,10
9,7844,TURNER,SALESMAN,7698.0,1981-09-08,1500.0,0.0,30


In [68]:
# 각 필드 특징 정보 (필드명, type, display_size,내부크기,정확도,scale,nullable)
#colname으로 쓴 아이들이 나옴
cursor.description     

[('EMPNO', <cx_Oracle.DbType DB_TYPE_NUMBER>, 5, None, 4, 0, 0),
 ('ENAME', <cx_Oracle.DbType DB_TYPE_VARCHAR>, 10, 10, None, None, 1),
 ('JOB', <cx_Oracle.DbType DB_TYPE_VARCHAR>, 9, 9, None, None, 1),
 ('MGR', <cx_Oracle.DbType DB_TYPE_NUMBER>, 5, None, 4, 0, 1),
 ('HIREDATE', <cx_Oracle.DbType DB_TYPE_DATE>, 23, None, None, None, 1),
 ('SAL', <cx_Oracle.DbType DB_TYPE_NUMBER>, 11, None, 7, 2, 1),
 ('COMM', <cx_Oracle.DbType DB_TYPE_NUMBER>, 11, None, 7, 2, 1),
 ('DEPTNO', <cx_Oracle.DbType DB_TYPE_NUMBER>, 3, None, 2, 0, 1)]

In [70]:
[row[0] for row in cursor.description]   # row[0]를 for문 앞에다  

['EMPNO', 'ENAME', 'JOB', 'MGR', 'HIREDATE', 'SAL', 'COMM', 'DEPTNO']

In [72]:
data_df.columns = [row[0] for row in cursor.description]   #***************

- 컬럼name 바꿔짐

In [73]:
data_df.columns 

Index(['EMPNO', 'ENAME', 'JOB', 'MGR', 'HIREDATE', 'SAL', 'COMM', 'DEPTNO'], dtype='object')

In [75]:
data_df.head()

,EMPNO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO
0,7369,SMITH,CLERK,7902.0,1980-12-17,800.0,NaN,20
1,7499,ALLEN,SALESMAN,7698.0,1981-02-20,1600.0,300.0,30
2,7521,WARD,SALESMAN,7698.0,1981-02-22,1250.0,500.0,30
3,7566,JONES,MANAGER,7839.0,1981-04-02,2975.0,NaN,20
4,7654,MARTIN,SALESMAN,7698.0,1981-09-28,1250.0,1400.0,30


In [76]:
data_df.tail()

,EMPNO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO
9,7844,TURNER,SALESMAN,7698.0,1981-09-08,1500.0,0.0,30
10,7876,ADAMS,CLERK,7788.0,1983-01-12,1100.0,NaN,20
11,7900,JAMES,CLERK,7698.0,1981-12-03,950.0,NaN,30
12,7902,FORD,ANALYST,7566.0,1981-12-03,3000.0,NaN,20
13,7934,MILLER,CLERK,7782.0,1982-01-23,1300.0,NaN,10


In [77]:
cursor.close()
conn.close()

# 3절. MariaDB 데이터베이스 연결

- anaconda prompt 실행 --> 입력 내용: pip install pymysql

In [79]:
import pymysql      #anaconda -->lib -->sitepackage 밑에 이게 없다는 듯

- workbench 실행-->   당시 설정 비번:mysql

In [81]:
conn = pymysql.connect(host="localhost",
                      port=3306,                            #port를 str(x)  int(o)로 입력
                      db="kimdb",user="root",passwd="mysql",
                      charset="utf8",autocommit=True)  #해당workbench 

In [83]:
cursor = conn.cursor()
sql = "select * from personal"
cursor.execute(sql)
result = cursor.fetchall()
for r in result:
    print(r)

(1001, 'bill', 'president', None, datetime.date(1989, 1, 10), 7000, None, 10)
(1111, 'smith', 'manager', 1001, datetime.date(1990, 12, 17), 1000, None, 10)
(1112, 'ally', 'salesman', 1116, datetime.date(1991, 2, 20), 1600, 500, 30)
(1113, 'word', 'salesman', 1116, datetime.date(1992, 2, 24), 1450, 300, 30)
(1114, 'james', 'manager', 1001, datetime.date(1990, 4, 12), 3975, None, 20)
(1116, 'johnson', 'manager', 1001, datetime.date(1991, 5, 1), 3550, None, 30)
(1118, 'martin', 'analyst', 1111, datetime.date(1991, 9, 9), 3450, None, 10)
(1121, 'kim', 'clerk', 1114, datetime.date(1990, 12, 8), 4000, None, 20)
(1123, 'lee', 'salesman', 1116, datetime.date(1991, 9, 23), 1200, 0, 30)
(1226, 'park', 'analyst', 1111, datetime.date(1990, 1, 3), 2500, None, 10)


In [84]:
personal = pd.DataFrame(result)      #여기서 pd       앞에 pandas를  as  pd 로 했음
personal.columns = [row[0] for row in cursor.description]   #colname 세팅됨
personal

,pno,pname,job,manager,startdate,pay,bonus,dno
0,1001,bill,president,NaN,1989-01-10,7000,NaN,10
1,1111,smith,manager,1001.0,1990-12-17,1000,NaN,10
2,1112,ally,salesman,1116.0,1991-02-20,1600,500.0,30
3,1113,word,salesman,1116.0,1992-02-24,1450,300.0,30
4,1114,james,manager,1001.0,1990-04-12,3975,NaN,20
5,1116,johnson,manager,1001.0,1991-05-01,3550,NaN,30
6,1118,martin,analyst,1111.0,1991-09-09,3450,NaN,10
7,1121,kim,clerk,1114.0,1990-12-08,4000,NaN,20
8,1123,lee,salesman,1116.0,1991-09-23,1200,0.0,30
9,1226,park,analyst,1111.0,1990-01-03,2500,NaN,10


In [85]:
cursor.close()
conn.close()

# 4절. 회원 입력/출력/삭제/검색 어플리케이션 
- ch10여기서는 data를 databse에       || (ch09 연습문제 실습형은  data를 txt파일에)
- SQLite이용, Oracle이용